In [1]:
!python3 -m pip install --upgrade pip setuptools wheel
# Try stock TF first; if it fails on Apple Silicon, comment this line and use the two lines below it
!python3 -m pip install --upgrade tensorflow pillow numpy matplotlib scikit-learn

# Apple Silicon fallback (uncomment if needed)
# !python3 -m pip install --upgrade tensorflow-macos==2.16.1 tensorflow-metal==1.1.0
# !python3 -m pip install --upgrade pillow numpy matplotlib scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os, random, shutil, json
from pathlib import Path
from typing import List
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

SHEEP_SRC  = Path("SheepFaceImages")   # root containing sheep breeds
BOVINE_SRC = Path("Indian_bovine_breeds")         # root containing bovine (cattle+buffalo) breeds

# Derived / working directories
WORK = Path("hierarchical_data")
SPECIES_ROOT = WORK / "species_binary"         # merged 'sheep' vs 'bovine' dataset
SHEEP_SPLIT  = WORK / "sheep_breeds"           # split per sheep breed (train/val/test)
BOVINE_SPLIT = WORK / "bovine_breeds"          # split per bovine breed (train/val/test)

ARTIFACTS = Path("artifacts_hier")
ARTIFACTS.mkdir(parents=True, exist_ok=True)

# Training config
IMG_SIZE = 224
BATCH = 32
EPOCHS_STAGE1 = 10
EPOCHS_STAGE2 = 12
FT_EPOCHS_STAGE1 = 6
FT_EPOCHS_STAGE2 = 8
LR = 1e-4
FT_LR = 1e-5

print("TensorFlow:", tf.__version__)
print("Sheep source exists:", SHEEP_SRC.exists())
print("Bovine source exists:", BOVINE_SRC.exists())


/Users/abhishekverghese/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow: 2.16.2
Sheep source exists: True
Bovine source exists: True


In [2]:
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def list_images(root: Path) -> list[Path]:
    return [p for p in root.rglob("*") if p.suffix.lower() in IMG_EXTS]

def safe_copy(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    token = f"{random.getrandbits(32):08x}"
    dst_final = dst.with_name(f"{dst.stem}_{token}{dst.suffix}")
    shutil.copy2(src, dst_final)

def split_indices(n, val_ratio=0.15, test_ratio=0.15):
    n_test = int(n * test_ratio)
    n_val = int(n * val_ratio)
    return slice(0, n_test), slice(n_test, n_test+n_val), slice(n_test+n_val, n)

def prepare_split_from_class_folders(source_root: Path, dest_root: Path,
                                     val_ratio=0.15, test_ratio=0.15, seed=42):
    random.seed(seed)
    if dest_root.exists():
        shutil.rmtree(dest_root)
    for split in ["train", "val", "test"]:
        (dest_root / split).mkdir(parents=True, exist_ok=True)

    for cls_dir in sorted([d for d in source_root.iterdir() if d.is_dir()]):
        imgs = list_images(cls_dir)
        random.shuffle(imgs)
        s_test, s_val, s_train = split_indices(len(imgs), val_ratio, test_ratio)
        for p in imgs[s_train]:
            safe_copy(p, dest_root/"train"/cls_dir.name/p.name)
        for p in imgs[s_val]:
            safe_copy(p, dest_root/"val"/cls_dir.name/p.name)
        for p in imgs[s_test]:
            safe_copy(p, dest_root/"test"/cls_dir.name/p.name)
    return True


In [3]:
def prepare_species_binary(sheep_root: Path, bovine_root: Path, dest_root: Path,
                           val_ratio=0.15, test_ratio=0.15, seed=42):
    random.seed(seed)
    if dest_root.exists():
        shutil.rmtree(dest_root)
    for split in ["train", "val", "test"]:
        (dest_root / split / "sheep").mkdir(parents=True, exist_ok=True)
        (dest_root / split / "bovine").mkdir(parents=True, exist_ok=True)

    sheep_imgs = list_images(sheep_root)
    bovine_imgs = list_images(bovine_root)
    random.shuffle(sheep_imgs); random.shuffle(bovine_imgs)

    def split(items):
        s_test, s_val, s_train = split_indices(len(items), val_ratio, test_ratio)
        return items[s_train], items[s_val], items[s_test]

    s_tr, s_va, s_te = split(sheep_imgs)
    b_tr, b_va, b_te = split(bovine_imgs)

    for p in s_tr: safe_copy(p, dest_root/"train"/"sheep"/p.name)
    for p in s_va: safe_copy(p, dest_root/"val"/"sheep"/p.name)
    for p in s_te: safe_copy(p, dest_root/"test"/"sheep"/p.name)

    for p in b_tr: safe_copy(p, dest_root/"train"/"bovine"/p.name)
    for p in b_va: safe_copy(p, dest_root/"val"/"bovine"/p.name)
    for p in b_te: safe_copy(p, dest_root/"test"/"bovine"/p.name)

    print("Stage-1 dataset built at:", dest_root)
    print("Sheep counts:", len(s_tr), len(s_va), len(s_te))
    print("Bovine counts:", len(b_tr), len(b_va), len(b_te))

# Build Stage-1 dataset
prepare_species_binary(SHEEP_SRC, BOVINE_SRC, SPECIES_ROOT, val_ratio=0.15, test_ratio=0.15)


Stage-1 dataset built at: hierarchical_data/species_binary
Sheep counts: 1176 252 252
Bovine counts: 7198 1542 1542


In [5]:
def make_ds_from_dir(root: Path, img_size=224, batch=BATCH, augment_train=True):
    train_ds = keras.utils.image_dataset_from_directory(
        root/"train", image_size=(img_size, img_size), batch_size=batch, shuffle=True
    )
    val_ds = keras.utils.image_dataset_from_directory(
        root/"val", image_size=(img_size, img_size), batch_size=batch, shuffle=False
    )
    test_ds = keras.utils.image_dataset_from_directory(
        root/"test", image_size=(img_size, img_size), batch_size=batch, shuffle=False
    )

    class_names = train_ds.class_names
    AUTOTUNE = tf.data.AUTOTUNE

    def norm(x, y):
        x = tf.cast(x, tf.float32) / 255.0
        return x, y

    aug = keras.Sequential(
        [
            keras.layers.RandomFlip("horizontal"),
            keras.layers.RandomRotation(0.1),
            keras.layers.RandomZoom(0.12),
            keras.layers.RandomContrast(0.1),
        ],
        name="augment",
    )

    if augment_train:
        train_ds = (
            train_ds
            .map(norm, num_parallel_calls=AUTOTUNE)
            .map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
            .prefetch(AUTOTUNE)
        )
    else:
        train_ds = train_ds.map(norm, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

    val_ds  = val_ds.map(norm, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    test_ds = test_ds.map(norm, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

    return train_ds, val_ds, test_ds, class_names


In [6]:
# Load Stage-1 dataset
sp_train, sp_val, sp_test, sp_classes = make_ds_from_dir(SPECIES_ROOT, IMG_SIZE, BATCH)
print("Stage-1 classes:", sp_classes)

# Build model
base_sp = keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg"
)
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = keras.applications.efficientnet.preprocess_input(inputs*255.0)
x = base_sp(x, training=False)
x = keras.layers.Dropout(0.3)(x)
outputs = keras.layers.Dense(len(sp_classes), activation="softmax")(x)
species_model = keras.Model(inputs, outputs)

species_model.compile(optimizer=keras.optimizers.Adam(LR),
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])

ckpt_sp = ARTIFACTS / "species_ckpt.keras"
callbacks_sp = [
    keras.callbacks.ModelCheckpoint(str(ckpt_sp), save_best_only=True, monitor="val_accuracy", mode="max"),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True)
]

print("Training Stage-1 (frozen)...")
species_model.fit(sp_train, validation_data=sp_val, epochs=EPOCHS_STAGE1, callbacks=callbacks_sp)

# Fine-tune last layers
base_sp.trainable = True
for lyr in base_sp.layers[:-40]:
    lyr.trainable = False

species_model.compile(optimizer=keras.optimizers.Adam(FT_LR),
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])

print("Fine-tuning Stage-1...")
species_model.fit(sp_train, validation_data=sp_val, epochs=FT_EPOCHS_STAGE1, callbacks=callbacks_sp)

# Evaluate + save
sp_loss, sp_acc = species_model.evaluate(sp_test)
print(f"Stage-1 Test accuracy: {sp_acc:.4f}")

species_model_path = ARTIFACTS / "species.model.keras"
species_model.save(species_model_path)
species_savedmodel_path = ARTIFACTS / "species.savedmodel"
species_model.export(species_savedmodel_path)
with open(ARTIFACTS / "species_labels.json", "w") as f:
    json.dump(sp_classes, f)
print("Saved Stage-1.")


Found 8374 files belonging to 2 classes.


2025-11-08 08:35:45.836050: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-08 08:35:45.836104: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-08 08:35:45.836108: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-11-08 08:35:45.836130: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-08 08:35:45.836146: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 1794 files belonging to 2 classes.
Found 1793 files belonging to 2 classes.
Stage-1 classes: ['bovine', 'sheep']
Training Stage-1 (frozen)...
Epoch 1/10


2025-11-08 08:35:47.850947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-08 08:35:59.725138: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-11-08 08:36:00.675108: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


 30/262 ━━━━━━━━━━━━━━━━━━━━ 6:54 2s/step - accuracy: 0.6734 - loss: 0.5858

...

262/262 ━━━━━━━━━━━━━━━━━━━━ 83s 315ms/step - accuracy: 0.9999 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 3.8381e-04
 1/57 ━━━━━━━━━━━━━━━━━━━━ 14s 262ms/step - accuracy: 0.9688 - loss: 0.0316

2025-11-08 09:27:41.041951: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile


57/57 ━━━━━━━━━━━━━━━━━━━━ 13s 220ms/step - accuracy: 0.9971 - loss: 0.0035
Stage-1 Test accuracy: 0.9989
INFO:tensorflow:Assets written to: artifacts_hier/species.savedmodel/assets


INFO:tensorflow:Assets written to: artifacts_hier/species.savedmodel/assets


Saved artifact at 'artifacts_hier/species.savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_244')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  14262910816: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  14262910640: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  14070813440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074010080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074010960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14070910464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074008672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074255840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074293936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074302880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14074291120: Te

In [7]:
# Create breed-level splits from your original sources.
# This assumes each source has subfolders per breed.
prepare_split_from_class_folders(SHEEP_SRC, SHEEP_SPLIT, val_ratio=0.15, test_ratio=0.15)
prepare_split_from_class_folders(BOVINE_SRC, BOVINE_SPLIT, val_ratio=0.15, test_ratio=0.15)

# Load them
sheep_train, sheep_val, sheep_test, sheep_breeds = make_ds_from_dir(SHEEP_SPLIT, IMG_SIZE, BATCH)
bovine_train, bovine_val, bovine_test, bovine_breeds = make_ds_from_dir(BOVINE_SPLIT, IMG_SIZE, BATCH)

print("Sheep breeds:", sheep_breeds)
print("Bovine breeds:", bovine_breeds)


Found 1176 files belonging to 4 classes.
Found 252 files belonging to 4 classes.
Found 252 files belonging to 4 classes.
Found 7237 files belonging to 41 classes.
Found 1522 files belonging to 41 classes.
Found 1522 files belonging to 41 classes.
Sheep breeds: ['Marino', 'Poll Dorset', 'Suffolk', 'White Suffolk']
Bovine breeds: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir', 'Guernsey', 'Hallikar', 'Hariana', 'Holstein_Friesian', 'Jaffrabadi', 'Jersey', 'Kangayam', 'Kankrej', 'Kasargod', 'Kenkatha', 'Kherigarh', 'Khillari', 'Krishna_Valley', 'Malnad_gidda', 'Mehsana', 'Murrah', 'Nagori', 'Nagpuri', 'Nili_Ravi', 'Nimari', 'Ongole', 'Pulikulam', 'Rathi', 'Red_Dane', 'Red_Sindhi', 'Sahiwal', 'Surti', 'Tharparkar', 'Toda', 'Umblachery', 'Vechur']


In [8]:
base_sh = keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg"
)
inp_sh = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x_sh = keras.applications.efficientnet.preprocess_input(inp_sh*255.0)
x_sh = base_sh(x_sh, training=False)
x_sh = keras.layers.Dropout(0.35)(x_sh)
out_sh = keras.layers.Dense(len(sheep_breeds), activation="softmax")(x_sh)
sheep_model = keras.Model(inp_sh, out_sh)

sheep_model.compile(optimizer=keras.optimizers.Adam(LR),
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

ckpt_sh = ARTIFACTS / "sheep_ckpt.keras"
cb_sh = [
    keras.callbacks.ModelCheckpoint(str(ckpt_sh), save_best_only=True, monitor="val_accuracy", mode="max"),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True)
]

print("Training Sheep breeds (frozen)...")
sheep_model.fit(sheep_train, validation_data=sheep_val, epochs=EPOCHS_STAGE2, callbacks=cb_sh)

base_sh.trainable = True
for lyr in base_sh.layers[:-40]:
    lyr.trainable = False

sheep_model.compile(optimizer=keras.optimizers.Adam(FT_LR),
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

print("Fine-tuning Sheep breeds...")
sheep_model.fit(sheep_train, validation_data=sheep_val, epochs=FT_EPOCHS_STAGE2, callbacks=cb_sh)

sh_loss, sh_acc = sheep_model.evaluate(sheep_test)
print(f"Sheep-breed Test accuracy: {sh_acc:.4f}")

sheep_model_path = ARTIFACTS / "sheep_breeds.model.keras"
sheep_model.save(sheep_model_path)
sheep_savedmodel_path = ARTIFACTS / "sheep_breeds.savedmodel"
sheep_model.export(sheep_savedmodel_path)
with open(ARTIFACTS / "sheep_breeds_labels.json", "w") as f:
    json.dump(sheep_breeds, f)
print("Saved Sheep-breed head.")


Training Sheep breeds (frozen)...
Epoch 1/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.3361 - loss: 1.3448 - val_accuracy: 0.4127 - val_loss: 1.1858
Epoch 2/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.6732 - loss: 0.9438 - val_accuracy: 0.6746 - val_loss: 0.8930
Epoch 3/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.7983 - loss: 0.6335 - val_accuracy: 0.7103 - val_loss: 0.7274
Epoch 4/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.8743 - loss: 0.4401 - val_accuracy: 0.7619 - val_loss: 0.6371
Epoch 5/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.9000 - loss: 0.3419 - val_accuracy: 0.8333 - val_loss: 0.4631
Epoch 6/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.9112 - loss: 0.2763 - val_accuracy: 0.8571 - val_loss: 0.3336
Epoch 7/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.9529 - loss: 0.1757 - val_accuracy: 0.9206 - val_loss: 0.2289
Epoch 8/12
37/37 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.9554 - loss: 0.1522

INFO:tensorflow:Assets written to: artifacts_hier/sheep_breeds.savedmodel/assets


Saved artifact at 'artifacts_hier/sheep_breeds.savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_498')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  15059042544: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  15059042896: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  14691103136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14526771968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14526772144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14779746608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14779748016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14526788176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14526804384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14526805440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1452679046

In [9]:
base_bv = keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg"
)
inp_bv = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x_bv = keras.applications.efficientnet.preprocess_input(inp_bv*255.0)
x_bv = base_bv(x_bv, training=False)
x_bv = keras.layers.Dropout(0.35)(x_bv)
out_bv = keras.layers.Dense(len(bovine_breeds), activation="softmax")(x_bv)
bovine_model = keras.Model(inp_bv, out_bv)

bovine_model.compile(optimizer=keras.optimizers.Adam(LR),
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])

ckpt_bv = ARTIFACTS / "bovine_ckpt.keras"
cb_bv = [
    keras.callbacks.ModelCheckpoint(str(ckpt_bv), save_best_only=True, monitor="val_accuracy", mode="max"),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True)
]

print("Training Bovine breeds (frozen)...")
bovine_model.fit(bovine_train, validation_data=bovine_val, epochs=EPOCHS_STAGE2, callbacks=cb_bv)

base_bv.trainable = True
for lyr in base_bv.layers[:-40]:
    lyr.trainable = False

bovine_model.compile(optimizer=keras.optimizers.Adam(FT_LR),
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])

print("Fine-tuning Bovine breeds...")
bovine_model.fit(bovine_train, validation_data=bovine_val, epochs=FT_EPOCHS_STAGE2, callbacks=cb_bv)

bv_loss, bv_acc = bovine_model.evaluate(bovine_test)
print(f"Bovine-breed Test accuracy: {bv_acc:.4f}")

bovine_model_path = ARTIFACTS / "bovine_breeds.model.keras"
bovine_model.save(bovine_model_path)
bovine_savedmodel_path = ARTIFACTS / "bovine_breeds.savedmodel"
bovine_model.export(bovine_savedmodel_path)
with open(ARTIFACTS / "bovine_breeds_labels.json", "w") as f:
    json.dump(bovine_breeds, f)
print("Saved Bovine-breed head.")


Training Bovine breeds (frozen)...
Epoch 1/12


2025-11-08 09:46:48.902714: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


 23/227 ━━━━━━━━━━━━━━━━━━━━ 11:17 3s/step - accuracy: 0.0313 - loss: 3.8331

2025-11-08 09:48:37.812281: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


 29/227 ━━━━━━━━━━━━━━━━━━━━ 10:13 3s/step - accuracy: 0.0326 - loss: 3.8144

...

17/48 ━━━━━━━━━━━━━━━━━━━━ 7s 229ms/step - accuracy: 0.7617 - loss: 0.9362

2025-11-08 11:34:58.867310: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


48/48 ━━━━━━━━━━━━━━━━━━━━ 12s 243ms/step - accuracy: 0.7554 - loss: 0.9335
Bovine-breed Test accuracy: 0.7503
INFO:tensorflow:Assets written to: artifacts_hier/bovine_breeds.savedmodel/assets


INFO:tensorflow:Assets written to: artifacts_hier/bovine_breeds.savedmodel/assets


Saved artifact at 'artifacts_hier/bovine_breeds.savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_742')
Output Type:
  TensorSpec(shape=(None, 41), dtype=tf.float32, name=None)
Captures:
  17021029888: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  17021029712: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  15694185920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17069691296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14780409280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  15893946032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14502158176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17602673648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17291657632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17291659392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  17602672

In [21]:
from PIL import Image

def load_image_rgb(path: str, img_size=IMG_SIZE):
    img = Image.open(path)
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize((img_size, img_size))
    arr = np.asarray(img).astype("float32") / 255.0
    return arr

def predict_hierarchical(image_path: str):
    # 1) Species
    arr = load_image_rgb(image_path)
    probs_sp = species_model.predict(np.expand_dims(arr, 0))[0]
    sp_idx = int(np.argmax(probs_sp))
    sp_label = sp_classes[sp_idx]
    sp_conf = float(probs_sp[sp_idx])

    # 2) Route to breed head
    if sp_label.lower() == "sheep":
        probs = sheep_model.predict(np.expand_dims(arr, 0))[0]
        idx = int(np.argmax(probs))
        breed = sheep_breeds[idx]
        conf = float(probs[idx])
        return {"species": sp_label, "species_conf": sp_conf,
                "breed_head": "sheep", "breed": breed, "breed_conf": conf}
    else:
        probs = bovine_model.predict(np.expand_dims(arr, 0))[0]
        idx = int(np.argmax(probs))
        breed = bovine_breeds[idx]
        conf = float(probs[idx])
        return {"species": sp_label, "species_conf": sp_conf,
                "breed_head": "bovine", "breed": breed, "breed_conf": conf}

# Example:
def classify_and_print(image_path: str):
    res = predict_hierarchical(image_path)
    print(f"Species: {res['species']}  (conf: {res['species_conf']:.2%})")
    print(f"Head:    {res['breed_head']}")
    print(f"Breed:   {res['breed']}  (conf: {res['breed_conf']:.2%})")
    return res

# Example
# Example
classify_and_print("/Users/abhishekverghese/Desktop/CowsheepClassifier/Indian_bovine_breeds/Hariana/Hariana_1.JPG")

# result


# result


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Species: bovine  (conf: 99.93%)
Head:    bovine
Breed:   Hariana  (conf: 59.62%)


{'species': 'bovine',
 'species_conf': 0.9992817044258118,
 'breed_head': 'bovine',
 'breed': 'Hariana',
 'breed_conf': 0.5961683988571167}

In [22]:
from pathlib import Path
import json, os

ROOT = Path(".")  # run this from COWSHEEPCLASSIFIER
ART = ROOT / "artifacts_hier"
HIER = ROOT / "hierarchical_data"

ART.mkdir(exist_ok=True, parents=True)

# Try to read labels jsons if they exist; else infer from train folders
def read_or_infer_labels(json_path: Path, fallback_dir: Path):
    if json_path.exists():
        return json.loads(json_path.read_text())
    # infer from subfolders under train
    train_dir = fallback_dir / "train"
    if train_dir.exists():
        labels = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
        return labels
    return []

router = {
    "species_model": str(ART / "species.model.keras"),
    "species_labels": ["bovine", "sheep"],  # stage-1 output order used during training
    "sheep_model": str(ART / "sheep_breeds.model.keras"),
    "sheep_labels": read_or_infer_labels(ART / "sheep_breeds_labels.json", HIER / "sheep_breeds"),
    "bovine_model": str(ART / "bovine_breeds.model.keras"),
    "bovine_labels": read_or_infer_labels(ART / "bovine_breeds_labels.json", HIER / "bovine_breeds"),
    "img_size": 224
}

(ART / "router_config.json").write_text(json.dumps(router, indent=2))
print("✅ wrote", ART / "router_config.json")
print("sheep labels:", len(router["sheep_labels"]))
print("bovine labels:", len(router["bovine_labels"]))


✅ wrote artifacts_hier/router_config.json
sheep labels: 4
bovine labels: 41
